In [63]:
import pandas as pd
# pd.set_option('display.width', 1000) 
# pd.set_option('display.max_colwidth', 1000) 
import numpy as np
from chromadb.utils.embedding_functions import OllamaEmbeddingFunction
from chromadb.config import Settings
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer
import re
import chromadb
import string
import uuid
from langdetect import detect

In [64]:
french_stopwords = set(stopwords.words('french'))

In [65]:
article = pd.read_csv("fstt-articles.csv")
clubs = pd.read_csv("fstt-clubs-info.csv")
dep = pd.read_csv("fstt-departements-info.csv")
formation = pd.read_csv("fstt-formation-initial.csv")
general_info = pd.read_csv("general_info.csv")
general_info.columns=["info", "fstt"]

In [66]:
clubs.rename(columns={'departement_info': 'club_description'}, inplace=True)

In [67]:
article.head(3)

,No,post_title,publish_date,post_content,post_link
0,1,CYBERSEC360° : PERSPECTIVES EN CYBERSÉCURITÉ A...,"mars 26, 2024",Un séminaire sera organisé le le samedi 30 Mar...,https://fstt.ac.ma/Portail2023/cybersec360-per...
1,2,PRÉSENTATION CONCOURS D’INNOVATION TDC,"mars 26, 2024",À la suite de l’appel à candidature au concour...,https://fstt.ac.ma/Portail2023/presentation-co...
2,3,SÉMINAIRE : WATER RESOURCES MANAGEMENT FOR SUS...,"mars 26, 2024","À l’occasion de la Journée mondiale de l’eau, ...",https://fstt.ac.ma/Portail2023/seminaire-water...


In [68]:
# Create a new column by merging all columns with a formatted string
article['merged_content'] = article.apply(
    lambda row: f"titre de post ou article : {row['post_title']} | contenu post ou article : {row['post_content']} | lien de post ou article : {row['post_link']}",
    axis=1
)

In [69]:
# Display the DataFrame
print(article['merged_content'][0])

titre de post ou article : CYBERSEC360° : PERSPECTIVES EN CYBERSÉCURITÉ AVANCÉE | contenu post ou article : Un séminaire sera organisé le le samedi 30 Mars 2024 (11h-13h) au profit des étudiants du Master Sécurité IT et Big Data et aux étudiants 2ème Année du cycle ingénieur LSI pour présenter et discuter les dernières tendances et les meilleures pratiques en matière de cybersécurité.
Le séminaire sera présenté par le professeur Hicham GIBET TANI, professeur à Faculté Polydisciplinaire de Larache, consultant et spécialiste en cybersécurité, sous le titre : CyberSec360°  Perspectives en Cybersécurité Avancée. Il propose une plongée approfondie dans les domaines complexes des concepts de cybersécurité avancée. | lien de post ou article : https://fstt.ac.ma/Portail2023/cybersec360-perspectives-en-cybersecurite-avancee/


In [70]:
# Create a new column by merging all columns with a formatted string
clubs['merged_content'] = clubs.apply(
    lambda row: f"nom du club : {row['club_name']} | description du club  : {row['club_description']} | lien du club : {row['club_link']}",
    axis=1
)

In [71]:
clubs['merged_content'][0]

'nom du club : GREENOLOGY | description du club  : Le Club de l’environnement de la FST de Tanger, Dénommé Greenology Club, est nouvellement créé et il a comme vocation de : – Transmettre aux étudiants, une conscience nationale éclairée, un patriotisme activé pour un environnement préservé. – Sensibiliser et rendre les étudiants de la FST de Tanger des combattants contre l’éparpillement de l’environnement. – Tous unis, bien convaincus que nos richesses environnementales très précieuses sont menacées, et qu’avec une conscience large du désastre écologique qui frappe à la porte de notre pays, on pourra gagner la bataille et sauver ce qu’il reste encore à sauver.\nla page facebook:\nhttps://www.facebook.com/greenology.club\nEmail de contact :\nResponsable d’activités 4saaddine@gmail.com | lien du club : https://fstt.ac.ma/Portail2023/greenology/'

In [72]:
# Remove newline characters from 'departement_info' column
dep['departement_info'] = dep['departement_info'].str.replace('\n', ' - ')

In [73]:
dep.head()

,No,departement_name,departement_info
0,1,GÉNIE INFORMATIQUE,Chef : Pr.EL BRAK Mohamed - Email : melb...
1,2,GÉNIE CHIMIQUE,Chef : Pr.CHABBI Mohamed - Email : mchabbi@uae...
2,3,SCIENCES DE LA TERRE,Chef : Pr.BOULAASSAL Hakim - Email : h.boulaas...
3,4,GÉNIE MÉCANIQUE,Chef : Pr.ELAYACHI Ilham - - Email :...
4,5,SCIENCES DE LA VIE,Chef : Pr.HASSANI ZERROUK Mounir - Email : mha...


In [74]:
# Create a new column by merging all columns with a formatted string
dep['merged_content'] = dep.apply(
    lambda row: f"nom de departement : {row['departement_name']} | le nom de Coordinnateur avec son email : {row['departement_info']}",
    axis=1
)

In [75]:
dep['merged_content'][0]

'nom de departement : GÉNIE INFORMATIQUE | le nom de Coordinnateur avec son email : Chef : Pr.EL BRAK Mohamed     - Email :   melbrak@uae.ac.ma'

In [76]:
formation.head()

,No,mst_name,formation_type,mst_link,mst_objectif,mst_program,mst_skills,mst_Coord
0,1,MST : Analyse Appliquée et Ingénierie Statistique,mst,https://fstt.ac.ma/Portail2023/mst-analyse-app...,Le Master Analyse Appliquée et Ingénierie Stat...,"{'Semestre 1': ['', ' Statistique mathématique...",Donner une formation mathématique et statistiq...,Coordinnateur pédagogique : Pr.EL HALIMI Rach...
1,2,MST : Bases Cellulaires et Moléculaires en Bio...,mst,https://fstt.ac.ma/Portail2023/mst-bases-cellu...,Le master BCMB vise la formation des spécialis...,"{'Semestre 1': ['', ' Techniques Expérimentale...","Secteur public : chercheur, responsable, ingén...",Coordinnateur pédagogique : Pr.Mohamed NHIRI ...
2,3,"MST : Environnement, Aquaculture et Développem...",mst,https://fstt.ac.ma/Portail2023/mst-environneme...,"Encore peu développée au Maroc, l’aquaculture ...","{'Semestre 1': ['', ' Écosystèmes aquatiques e...",NaN,Coordinnateur pédagogique : Pr.Hassan Er-RAIOU...
3,4,MST : Géoressources Energétiques et Réservoirs,mst,https://fstt.ac.ma/Portail2023/mst-georessourc...,L’activité accrue dans la recherche en géoscie...,"{'Semestre 1': ['', ' Géodynamique et Analyse ...",NaN,Coordinnateur pédagogique : Pr.Mohamed Najib Z...
4,5,MST : Génie Civil,mst,https://fstt.ac.ma/Portail2023/mst-genie-civil/,Le programme vise la formation des cadres en G...,"{'Semestre 1': ['', ' Mathématiques Pour l’Ing...",NaN,Coordinnateur pédagogique : Pr.Dkiouak Rachid ...


In [77]:
# Create a new column by merging all columns with a formatted string
formation['merged_content'] = formation.apply(
    lambda row: f"nom de formation : {row['mst_name']} | type de formation  : {row['formation_type']} | objective de formation  : {row['mst_objectif']} | programme de formation : {row['mst_program']} | Coordinnateur de formation : {row['mst_Coord']} | link de formation : {row['mst_link']} |",
    axis=1
)

In [78]:
formation['merged_content'][0]

"nom de formation : MST : Analyse Appliquée et Ingénierie Statistique | type de formation  : mst | objective de formation  : Le Master Analyse Appliquée et Ingénierie Statistique (MAAIS) de l'Université Abdelmalek Essaadi a pour vocation d'offrir des parcours qui trouvent leurs bases dans une approche pluridisciplinaire. Au département de mathématiques on propose aux étudiants d’acquérir une double compétence en mathématiques et en statistiques appliquées s’appuyant sur un tronc commun développant des concepts et des outils de base. Pour une approche pluridisciplinaire des problèmes liés aux applications, les mathématiques et les statistiques appliquées ne peuvent que former des étudiants pour la recherche et les activités professionnelles de haut niveau avec un bon bagage mathématique. La spécialisation se ferait au terme du second semestre (S2) selon le choix des étudiants. En plus une bonne formation en tronc commun solide permettra de mieux optimiser les ressources humaines surtout

In [79]:
general_info.head()

,info,fstt
0,PRÉSENTATION GÉNÉRALE sur FST Tanger :\nFacult...,PRÉSENTATION GÉNÉRALE sur FST Tanger
1,La FST Tanger regroupe actuellement une trenta...,les départements de FSTT
2,La FST de Tanger a pour mission de dispenser l...,les diplômes de FST Tanget
3,Le nom du Doyen de FSTT est Pr.MUSTAPHA DIANI,le Doyen de FSTT
4,La Faculté des sciences et techniques de Tange...,La Faculté des sciences et techniques de Tange...


In [80]:
# Remove newline characters from 'departement_info' column
general_info['info'] = general_info['info'].str.replace('\n', '  ')

In [81]:
def is_french(text):
    try:
        return detect(text) == 'fr'
    except:
        return False
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False
def is_arabic(text):
    try:
        return detect(text) == 'ar'
    except:
        return False
def preprocessing(text):
    # Convert to string and lowercase
    text = str(text).lower()
    
    # Remove URLs
    # text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove mentions
    text = re.sub(r'@\w+', ' ', text)

    text = re.sub(r'\xa0', ' ', text)
    
    # Remove \n
    text = re.sub(r'\n', ' ', text)
    
    # Remove specific unwanted characters
    text = re.sub(r'«|»|“|”|’|‘', ' ', text)
    if not is_french(text):
        return ''

        # Tokenization
    """tokens = sent_tokenize(text, language="french")
    print("Tokenization :" ,tokens)"""
    
        
    # Remove punctuation
    # text = text.translate(str.maketrans("", "", string.punctuation))
    
    # # Tokenization using RegexpTokenizer
    # tokenizer = RegexpTokenizer(r'\w+')
    # tokens = tokenizer.tokenize(text)
    
    # tokens = word_tokenize(text, language="french")
    # tokens = [word for word in tokens if word not in french_stopwords]
    
    
    # Stemming
    # stemmer =nltk.stem.snowball.FrenchStemmer()
    # tokens_stemmed = [stemmer.stem(word) for word in tokens]
    
    # Join tokens back into a single string
    return text

In [82]:
article["merged_content"] = article["merged_content"].apply(preprocessing)
clubs["merged_content"] = clubs["merged_content"].apply(preprocessing)
formation["merged_content"] = formation["merged_content"].apply(preprocessing)
dep["merged_content"] = dep["merged_content"].apply(preprocessing)
general_info['info'] = general_info['info'].apply(preprocessing)

In [83]:
article["merged_content"][0]

'titre de post ou article : cybersec360° : perspectives en cybersécurité avancée | contenu post ou article : un séminaire sera organisé le le samedi 30 mars 2024 (11h-13h) au profit des étudiants du master sécurité it et big data et aux étudiants 2ème année du cycle ingénieur lsi pour présenter et discuter les dernières tendances et les meilleures pratiques en matière de cybersécurité. le séminaire sera présenté par le professeur hicham gibet tani, professeur à faculté polydisciplinaire de larache, consultant et spécialiste en cybersécurité, sous le titre : cybersec360°  perspectives en cybersécurité avancée. il propose une plongée approfondie dans les domaines complexes des concepts de cybersécurité avancée. | lien de post ou article : https://fstt.ac.ma/portail2023/cybersec360-perspectives-en-cybersecurite-avancee/'

In [84]:
general_info['info'][0]

"présentation générale sur fst tanger :  faculté des sciences et techniques tanger : ( fst tanger ou fstt tanger ), est un établissement universitaire à caractère scientifique et technique, la fst est destinée à s'intégrer dans le pôle technologique et industriel de la région pour être une pépinière de techniciens et de cadres de haut niveau capables de servir de courroie de transmission entre le technicien supérieur et l'ingénieur concepteur.      créée en 1995, la fst de tanger est un des huit établissements de l université abdelmalek essaâdi. elle regroupe actuellement une trentaine de programmes d'études repartis sur quatre cycles offerts par neuf  départements : sciences de la vie, sciences de la terre, génie chimique, physique, mathématiques, génie informatique, génie electrique et génie mécanique et  de langues.    derrière ses programmes d enseignement et de recherche, totalement inscrits dans le courant actuel des sciences et techniques, la fst de tanger compte avec un corps p

In [85]:
article.drop(columns="No", inplace=True)
clubs.drop(columns="No", inplace=True)
formation.drop(columns="No", inplace=True)
dep.drop(columns="No", inplace=True)

In [86]:
# Configure the ChromaDB client with persistence
persist_directory = "/home/idriss/Desktop/chroma_db"
client = chromadb.HttpClient(host='localhost', port=8000)

In [87]:
client.delete_collection(name="text_embeddings")

In [88]:
import chromadb.utils.embedding_functions as embedding_functions
huggingface_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="sentence-transformers/multi-qa-mpnet-base-dot-v1"
)
# Create a collection in ChromaDB with the OllamaEmbeddingFunction
collection = client.create_collection(name="text_embeddings",embedding_function=huggingface_ef)

In [89]:
import uuid

def process_and_store_embeddings(dataframe, column_names, collection):
    ids = []
    metadatas = []
    documents = []

    for idx, row in dataframe.iterrows():
        doc_metadata = {}
        for key, value in row.items():
            if key not in column_names:
                doc_metadata[key] = value  # Leave non-string fields as is
        
        for column in column_names:
            sentence = row[column]
            if sentence is not None and sentence != '':
                ids.append(str(uuid.uuid1()))
                metadatas.append(doc_metadata)
                documents.append(sentence)
    collection.add(
        ids=ids,
        #embeddings=embeddings,
        metadatas=metadatas,
        documents=documents
    )

# Assuming article, clubs, formation DataFrames are already preprocessed
# Store embeddings, metadata, and documents from each DataFrame
process_and_store_embeddings(article, ["merged_content"], collection)
process_and_store_embeddings(clubs, ["merged_content"], collection)
process_and_store_embeddings(formation, ["merged_content"], collection)
process_and_store_embeddings(dep, ["merged_content"], collection)
process_and_store_embeddings(general_info, ["info"], collection)

print("Embeddings, metadata, and documents stored in ChromaDB successfully.")


Embeddings, metadata, and documents stored in ChromaDB successfully.


In [90]:
collection.peek()

{'ids': ['0849e048-2286-11ef-88ee-803049a3c985',
  '0849ead4-2286-11ef-88ee-803049a3c985',
  '0849eed0-2286-11ef-88ee-803049a3c985',
  '0849f22c-2286-11ef-88ee-803049a3c985',
  '0849f56a-2286-11ef-88ee-803049a3c985',
  '0849f8da-2286-11ef-88ee-803049a3c985',
  '0849fc4a-2286-11ef-88ee-803049a3c985',
  '0849ffb0-2286-11ef-88ee-803049a3c985',
  '084a02b2-2286-11ef-88ee-803049a3c985',
  '084a05d2-2286-11ef-88ee-803049a3c985'],
 'embeddings': [[-0.011127641424536705,
   0.03783295303583145,
   -0.22166313230991364,
   0.22768664360046387,
   -0.1216762438416481,
   -0.35267892479896545,
   0.5094408988952637,
   -0.10101374238729477,
   -0.1838381290435791,
   0.024305222555994987,
   0.29688236117362976,
   -0.01747998595237732,
   -0.09953851252794266,
   0.44221773743629456,
   0.1075296625494957,
   -0.06938933581113815,
   0.03056485392153263,
   0.14751997590065002,
   -0.26605358719825745,
   0.18113815784454346,
   0.03299052640795708,
   0.008740769699215889,
   -0.434381097555160

In [91]:
# Load the existing collection
collection = client.get_collection(name="text_embeddings",embedding_function=huggingface_ef)

# Perform a query
query = "quel est les formation de la fstt ?"
results = collection.query(
    query_texts=[query],
    n_results=4
)

for result in results['documents']:
    print(result)


["nom de formation : mathématiques et applications | type de formation  : lst | objective de formation  : acquérir une double compétence mathématiques et informatique grâce à laquelle ils aborderont les problèmes de modélisation et de simulation scientifique sous tous leurs aspects, de la conception à la mise en oeuvre numérique. l accent est donc mis à la fois sur des techniques d ingénierie mathématiques et sur les langages de programmation et les outils algorithmiques. | programme de formation : {'semestre 5': ['', ' topologies dans les espaces métriques', ' intégration', ' probabilités avancées', ' recherche opérationnelle', ' concepts de base de programmation orientée c', ' analyse numérique matricielle'], 'semestre 6': ['', ' calcul différentiel et équations différentielles', ' topologie générale et distributions', ' analyse numérique des équations différentielles', ' projet de fin d études']} | coordinnateur de formation : coordinnateur pédagogique : pr.lahrouz adil           : 